In [12]:
input_small = """########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

<^^>>>vv<v>>v<<"""

input_medium = """##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^"""

with open("input.txt", "r") as f:
    input_large = f.read()

In [13]:
import numpy as np


def parse_input(input_text: str):
    table = []
    lines = [x.strip() for x in input_text.splitlines()]
    start_pos = None
    for i, line in enumerate(lines):
        if i == 0:
            assert set(line) == {"#"}
        elif set(line) == {"#"}:
            assert lines[i + 1].strip() == ""
            movements = "".join(lines[i + 2 :]).strip()
            break
        else:
            assert "#" == line[0]
            assert "#" == line[-1]
            subline = line[1:-1]
            if "@" in subline:
                start_pos = (i - 1, subline.index("@"))
            table.append(list(subline))
    return np.array(table), movements, start_pos


from typing import Literal


def table_iterator(table, start_pos, direction: Literal["<", "^", ">", "v"]):
    row, col = start_pos
    max_row, max_col = table.shape

    if direction == ">":
        while col < max_col:
            yield (row, col)
            col += 1
    elif direction == "<":
        while col >= 0:
            yield (row, col)
            col -= 1
    elif direction == "v":
        while row < max_row:
            yield (row, col)
            row += 1
    elif direction == "^":
        while row >= 0:
            yield (row, col)
            row -= 1


def move_logic(table, start_pos, dir: Literal["<", "^", ">", "v"]):
    iterator = table_iterator(table, start_pos, dir)
    assert start_pos == next(iterator)  # get start pos
    assert table[start_pos] == "@"

    for idx, pos in enumerate(iterator):
        if idx == 0:
            first_pos = pos
        val = table[pos]
        if val == "O":
            continue
        elif val == "#":
            return start_pos
        elif val == ".":
            table[pos] = "O"
            table[first_pos] = "@"
            table[start_pos] = "."
            return first_pos
    # hits the border w/o .
    return start_pos


import pandas as pd
from IPython.display import HTML


def display_colored_table(array):
    df = pd.DataFrame(array)

    def color_cells(val):
        if val == "@":
            return "background-color: red"
        elif val == "#":
            return "background-color: blue"
        elif val in "O[]":
            return "background-color: green"
        return ""

    styled_df = df.style.applymap(color_cells)
    display(HTML(styled_df.to_html()))
    return None


def apply_movements(table, start_pos, movements, display: bool = False):
    table = table.copy()
    if display:
        display_colored_table(table)

    pos = start_pos
    for mov in movements:
        new_pos = move_logic(table, pos, mov)

        if display:
            print(f"{mov}: {pos} -> {new_pos}")
            display_colored_table(table)
        pos = new_pos
    return table


def get_points(table):
    positions = np.where(table == "O")
    total_points = 0
    for row, col in zip(*positions):
        point = 100 * (row + 1) + (col + 1)
        total_points += point
        # print(row, col)
    return total_points


In [14]:
table, movements, start_pos = parse_input(input_large)
# print(movements)

new_table = apply_movements(table, start_pos, movements, display=False)
get_points(new_table)

1476771

## Part b

In [66]:
# If the tile is #, the new map contains ## instead.
# If the tile is O, the new map contains [] instead.
# If the tile is ., the new map contains .. instead.
# If the tile is @, the new map contains @. instead.

conversion = {"#": "##", "O": "[]", ".": "..", "@": "@."}


def parse_input2(input_text: str):
    table = []
    lines = [x.strip() for x in input_text.splitlines()]
    start_pos = None
    for i, line in enumerate(lines):
        if i == 0:
            assert set(line) == {"#"}
        elif set(line) == {"#"}:
            assert lines[i + 1].strip() == ""
            movements = "".join(lines[i + 2 :]).strip()
            break
        else:
            assert "#" == line[0]
            assert "#" == line[-1]
            subline = line[1:-1]
            subline_expanded = "".join([conversion[char] for char in subline])
            if "@" in subline_expanded:
                start_pos = (i - 1, subline_expanded.index("@"))
            table.append(list(subline_expanded))
    return np.array(table), movements, start_pos


def next_pos(table, start_pos, dir: ["^", "v"]):
    row, col = start_pos
    max_row, max_col = table.shape
    if dir == "^":
        next_pos = (row - 1, col)
        if next_pos[0] >= 0:
            return next_pos
    elif dir == "v":
        next_pos = (row + 1, col)
        if next_pos[0] < max_row:
            return next_pos
    elif dir == "<":
        next_pos = (row, col - 1)
        if next_pos[1] >= 0:
            return next_pos
    elif dir == ">":
        next_pos = (row, col + 1)
        if next_pos[1] < max_col:
            return next_pos
    else:
        raise ValueError
    return None


def move_logic2(table, start_pos, dir: Literal["<", "^", ">", "v"]):
    if dir in ("<", ">"):
        iterator = table_iterator(table, start_pos, dir)
        assert start_pos == next(iterator)  # get start pos
        assert table[start_pos] == "@"

        box_positions = []
        for idx, pos in enumerate(iterator):
            if idx == 0:
                first_pos = pos
            val = table[pos]
            if val in ("[", "]"):
                box_positions.append(pos)
            elif val == "#":
                return start_pos
            elif val == ".":
                table[start_pos] = "."

                # move boxes to the left/right
                new_box_positions = box_positions[1:] + [pos]
                for i, box_pos in enumerate(new_box_positions):
                    if (i % 2 == 0 and dir == ">") or (i % 2 == 1 and dir == "<"):
                        table[box_pos] = "["
                    else:
                        table[box_pos] = "]"
                table[first_pos] = "@"
                return first_pos
        # hits the border w/o .
    elif dir in ("^", "v"):
        assert table[start_pos] == "@"
        first_pos = next_pos(table, start_pos, dir)
        stack = [first_pos]
        is_movable = True
        box_positions = []
        first_box = None
        while len(stack) > 0:
            pos = stack.pop()
            # boundry of table
            if pos is None:
                is_movable = False
                break
            val = table[pos]
            # hit a block
            if val == "#":
                is_movable = False
                break
            elif val in ("[", "]"):
                displacement = 1 if val == "[" else -1
                partner_pos = (pos[0], pos[1] + displacement)
                if first_box is None:
                    first_box = (pos, partner_pos)
                    # ([ pos, ] pos)
                    first_box = sorted(first_box, key=lambda x: x[1])
                box_positions.append(pos)
                box_positions.append(partner_pos)
                stack.append(next_pos(table, pos, dir))
                stack.append(next_pos(table, partner_pos, dir))
        if not is_movable:
            return start_pos
        else:
            table[start_pos] = "."
            # need to be reversed to don't correcly propagate
            reverse = dir == "v"

            for box_pos in sorted(
                set(box_positions), key=lambda x: x[0], reverse=reverse
            ):
                table[next_pos(table, box_pos, dir)] = table[box_pos]
                table[box_pos] = "."
            if first_box is not None:
                for box_pos in first_box:
                    table[box_pos] = "."
            table[first_pos] = "@"
            return first_pos
    else:
        raise ValueError(f"{dir=} not implemented")

    return start_pos


from collections import Counter


def apply_movements2(table, start_pos, movements, display: bool = False):
    table = table.copy()
    if display:
        display_colored_table(table)

    pos = start_pos
    for mov in movements:
        counts_before = Counter(table.flatten())
        new_pos = move_logic2(table, pos, mov)
        counts_after = Counter(table.flatten())
        assert counts_before == counts_after
        if display:
            print(f"{mov}: {pos} -> {new_pos}")
            display_colored_table(table)
        pos = new_pos
    return table


def get_points2(table):
    positions = np.where(table == "[")
    total_points = 0
    for row, col in zip(*positions):
        point = 100 * (row + 1) + (col + 2)
        total_points += point
        # print(row, col)
    return total_points


input_small_2 = """#######
#...#.#
#.....#
#..OO@#
#..O..#
#.....#
#######

<vv<<^^<<^^"""
table, movements, start_pos = parse_input2(input_large)

# display_colored_table(table)
final_table = apply_movements2(table, start_pos, movements, display=False)
points = get_points2(final_table)
print(f"Points: {points}")
# display_colored_table(final_table)

Points: 1468005


Counter({'.': 83, '[': 21, ']': 21, '#': 2, '@': 1})

## Random Linear Algebra Stuff

In [16]:
def project_vector_to_subspace(v, A):
    """
    Projects a vector v onto the subspace defined by the columns of matrix A.

    Parameters:
    v (numpy.ndarray): The vector to project, shape (n, ).
    A (numpy.ndarray): The matrix whose columns span the subspace, shape (n, m).

    Returns:
    numpy.ndarray: The projection of v onto the subspace.
    """
    # Ensure inputs are numpy arrays
    v = np.asarray(v)
    A = np.asarray(A)

    # Check dimensions for compatibility
    if A.shape[0] != v.shape[0]:
        raise ValueError("The dimensions of vector v and matrix A are incompatible.")

    # Calculate the projection
    # Projection formula: A (A^T A)^(-1) A^T v
    AtA = A.T @ A
    AtA_inv = np.linalg.inv(AtA)  # Inverse of A^T A
    projection = A @ AtA_inv @ A.T @ v
    return projection


# Example usage
if __name__ == "__main__":
    # Define a vector and a subspace
    v = np.array([1, 2, 0, 5])  # Vector to project
    A = np.array(
        [
            [1, -3],  # Subspace spanned by columns of A
            [1, 1],
            [1, -3],
            [1, 5],
        ]
    )

    # Project vector
    projection = project_vector_to_subspace(v, A)
    print("Projection of v onto the subspace:", projection.round(2))


# Example usage
if __name__ == "__main__":
    # Define a vector and a subspace
    v = np.array([1, 2, 0, 5])  # Vector to project
    A = np.array(
        [
            [1, 1],  # Subspace spanned by columns of A
            [1, 3],
            [1, 1],
            [1, 5],
        ]
    )

    # Project vector
    projection = project_vector_to_subspace(v, A)
    print("Projection of v onto the subspace:", projection.round(2))

Projection of v onto the subspace: [0.36 2.55 0.36 4.73]
Projection of v onto the subspace: [0.36 2.55 0.36 4.73]


In [17]:
import numpy as np


def least_squares_solution(A, b):
    """
    Solves the least squares problem to find the vector x that minimizes ||Ax - b||^2.

    Parameters:
    A (numpy.ndarray): The matrix defining the linear system, shape (m, n).
    b (numpy.ndarray): The target vector, shape (m, ).

    Returns:
    numpy.ndarray: The vector x that minimizes ||Ax - b||^2.
    """
    # Ensure inputs are numpy arrays
    A = np.asarray(A)
    b = np.asarray(b)

    # Check dimensions for compatibility
    if A.shape[0] != b.shape[0]:
        raise ValueError("The number of rows in A must match the size of b.")

    # Solve the normal equation: (A^T A) x = A^T b
    AtA = A.T @ A
    Atb = A.T @ b
    x = np.linalg.solve(AtA, Atb)  # Solve the linear system
    return x


# Example usage
if __name__ == "__main__":
    # Define the matrix A and vector b
    A = np.array([[1, 1], [3, 1], [1, 1], [5, 1]])
    b = np.array([1, 2, 0, 5])  # Target vector

    # Compute least squares solution
    x = least_squares_solution(A, b)
    print("Least squares solution:", x.round(2))
    print(np.array([12 / 11, -8 / 11]).round(2))

Least squares solution: [ 1.09 -0.73]
[ 1.09 -0.73]


In [18]:
from sympy import Matrix


def compute_rref(matrix):
    """
    Computes the Reduced Row Echelon Form (RREF) of a matrix using sympy.

    Parameters:
    matrix (list or numpy.ndarray): The input matrix.

    Returns:
    tuple: A tuple containing:
        - The RREF of the matrix as a sympy Matrix.
        - A list of pivot columns.
    """
    sympy_matrix = Matrix(matrix)  # Convert to a sympy Matrix
    rref_matrix, pivots = sympy_matrix.rref()  # Compute RREF
    return rref_matrix, pivots


# Example usage
if __name__ == "__main__":
    # Define a matrix
    A = [[-8, 2, 1, 0, 0], [4, -2, 0, 1, 0], [0, 0, 0, 0, 1]]

    # Compute RREF
    rref_matrix, pivots = compute_rref(A)
    print("Reduced Row Echelon Form (RREF):")
    print(rref_matrix)
    print("Pivot columns:", pivots)

Reduced Row Echelon Form (RREF):
Matrix([[1, 0, -1/4, -1/4, 0], [0, 1, -1/2, -1, 0], [0, 0, 0, 0, 1]])
Pivot columns: (0, 1, 4)


In [19]:
u = [6, -5, 8]
v = [10, -5, 0]

u = np.array(u)
v = np.array(v)


costheta = np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
print(costheta)
theta = np.arccos(costheta) * 180 / np.pi
print(theta)

0.6799999999999999
47.15635695640367
